### Import Required Package

In [1]:
import pandas as pd
import talib
import numpy as np
import json
import os
os.chdir('/home/jovyan/individualized-indicator')
import yaml
import glob
import pandas as pd
import gcsfs
fs = gcsfs.GCSFileSystem(project="dst-dev2021")
from backtest.simulated_backtest_pkl import *
from datetime import datetime
from dateutil.relativedelta import relativedelta

### Import Tidal

In [2]:
import tidal as td

### Initialize Plumber

In [3]:
from pathlib import Path
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

PLUMBER_HOST = "https://dev-api.ddt-dst.cc/api/plumber/"
with open(f'{str(Path.home())}/.config/gcloud/application_default_credentials.json') as plumber_token:
    token = json.load(plumber_token)

In [4]:
today = datetime.today()
one_month_ago = today - relativedelta(months=1)
formatted_date = one_month_ago.strftime('%Y-%m-%d')

total_data = pd.read_parquet(
    f"{PLUMBER_HOST}stocks/tw/ohlcv",
    storage_options={
        "gcp-token": json.dumps(token),
        "start-date": formatted_date,
        "tickers": ",".join([stock for stock in ['Y9999']]),
    },
)

latest_data = total_data.reset_index()
latest_data = latest_data.iloc[-1][1].strftime("%Y-%m-%d")
latest_data

'2024-11-04'

### Simulated_price

In [5]:
ind_start = '2021-11-01'
ind_end = latest_data
# ind_end = datetime.today().strftime('%Y-%m-%d')
day_start = '2024-04-01'
day_end = '2024-08-30'
test_start = '2024-09-02'
# 取得當天的日期
test_end = ind_end
train_season = '2024_06_01'
emb_length = 'embeddings_length200_2024_06_01.npy'

etl = recommend_stock(emb_length, ind_start, ind_end, day_start, day_end, test_start, test_end, train_season)

indicator_path_r = 'jeff-stock-wise/indicator/'+ train_season + '/indicator_chose.csv'
        
with fs.open(indicator_path_r, 'r') as f:
    indicator_top_list = pd.read_csv(f, index_col=0)

TOP_k = len(indicator_top_list)

# 回測
input_backtest_table = Input_backtest_table(etl,'top_'+str(TOP_k), test_start, test_end, train_season)
input_backtest_table_calculate = input_backtest_table.calculate()

ok


100%|██████████| 324/324 [00:28<00:00, 11.21it/s]


### Load market data and compute technical indicators

In [6]:
gcs_path = 'jeff-stock-wise/simulated/'

start_date = "2024-10-01"
end_date = latest_data

# 初始化一個空的字典來存儲所有讀取的 DataFrame
dataframes = {}

# 遍歷目錄中的所有檔案
for filename in fs.ls(gcs_path):
    if filename.endswith('.pkl'):
        # 組合完整的檔案路徑
        with fs.open(filename, 'rb') as f:
        # 讀取 .pkl 檔案
            df = pd.read_pickle(f)
        
        # 使用檔案名稱（不包括副檔名）作為字典的鍵
        variable_name = filename.split('_')[2]
        
        # 將 DataFrame 儲存到字典中
        dataframes[variable_name] = df

keys_list = list(dataframes.keys())
# 將字符串轉換為 datetime 格式，然後進行排序
sorted_dates = sorted(pd.to_datetime(keys_list))
keys_list = [date.strftime('%Y-%m-%d') for date in sorted_dates]

total_df = pd.DataFrame()

for j in range(0,len(keys_list)):
    df_name = keys_list[j]
    selected_df = dataframes[df_name]    
    selected_df = selected_df.reset_index()
    stock_list = pd.DataFrame(selected_df.iloc[:,0].unique())
    total_df = pd.concat([total_df, stock_list], ignore_index=True)
    
unique_values_list = list(total_df.iloc[:,0].unique())
sorted_list = sorted(unique_values_list)

total_data = pd.read_parquet(
    f"{PLUMBER_HOST}stocks/tw/ohlcv",
    storage_options={
        "gcp-token": json.dumps(token),
        "start-date": start_date,
        "end-date": end_date,
        "tickers": ",".join([stock for stock in sorted_list]),
    },
)

total_data.index = total_data.index.set_levels(
    pd.to_datetime(total_data.index.levels[1]),level=1)

total_data.rename_axis(index={
    'ticker': 'instrument'
}, inplace=True)

total_data['new'] = -1
total_data.reset_index(inplace=True)

def filter_by_year_month_12(filter_data, select_year, select_mon):
    # 計算當前篩選年份和下一個年份
    next_year = select_year + 1
    
    # 構建篩選條件：當前年12月、次年1月、次年2月
    filter_data = filter_data[
        ((filter_data['datetime'].dt.year == next_year) & (filter_data['datetime'].dt.month == 1)) |
        ((filter_data['datetime'].dt.year == next_year) & (filter_data['datetime'].dt.month == 2)) |
        ((filter_data['datetime'].dt.year == next_year) & (filter_data['datetime'].dt.month == 3))
    ]
    
    return filter_data

concat_df = pd.DataFrame()

# for i in range(0,len(keys_list)-1):
for i in range(0,len(keys_list)):
    df_name = keys_list[i]
    selected_df = dataframes[df_name]
    selected_df.reset_index(inplace=True)
    stock_df = selected_df.iloc[:,0]
    stock_df_unique = list(stock_df.unique())
    result = list(set(sorted_list) - set(stock_df_unique))
    filtered_df = total_data[total_data['instrument'].isin(result)]
    filtered_df = filtered_df.rename(columns={'volume': 'vol'})
    selected_df = pd.concat([selected_df, filtered_df], ignore_index=True)
    
    select_year = int(df_name.split('-')[0])
    select_mon = int(df_name.split('-')[1])
    select_mons = [(select_mon+1)%12, (select_mon+2)%12, (select_mon+3)%12]
    select_mons = [12 if mon == 0 else mon for mon in select_mons]
    
    selected_df['datetime'] = pd.to_datetime(selected_df['datetime'])
    # 過濾出所有6月的交易資料
    filter_data = selected_df[selected_df['datetime'].dt.month.isin(select_mons)]

    # 使用時，根據 select_year 和 select_mon 動態篩選數據
        
    if select_mon == 12:
        filter_data = filter_by_year_month_12(filter_data, select_year, select_mon) 

    else:
        filter_data = filter_data[filter_data['datetime'].dt.year == select_year]
    
    concat_df = pd.concat([concat_df, filter_data], ignore_index=True)
    
# i = len(keys_list)-1
# df_name = keys_list[i]
# selected_df = dataframes[df_name]
# selected_df.reset_index(inplace=True)
# stock_df = selected_df.iloc[:,0]
# stock_df_unique = list(stock_df.unique())
# result = list(set(sorted_list) - set(stock_df_unique))
# filtered_df = total_data[total_data['instrument'].isin(result)]
# filtered_df = filtered_df.rename(columns={'volume': 'vol'})
# selected_df = pd.concat([selected_df, filtered_df], ignore_index=True)
# concat_df = pd.concat([concat_df, selected_df], ignore_index=True)

concat_df.sort_values(by=['instrument', 'datetime'], inplace=True)
concat_df.reset_index(drop=True, inplace=True)
row = np.where(concat_df.iloc[:,0] == '6251')[0]
concat_df = concat_df.drop(index=row)
concat_df = concat_df.reset_index(drop=True)
row = np.where(concat_df.iloc[:,0] == '8406')[0]
concat_df = concat_df.drop(index=row)
concat_df = concat_df.reset_index(drop=True)
row = np.where(concat_df.iloc[:,0] == '6548')[0]
concat_df = concat_df.drop(index=row)
concat_df = concat_df.reset_index(drop=True)
quote_data = concat_df.set_index(['instrument', 'datetime'])
quote_data.columns.values[4] = 'volume'
quote_data
# quote_data.to_csv('./quote_data.csv')

open    high     low   close      volume       new
instrument datetime                                                        
1101       2024-10-01   33.90   33.90   33.45   33.70  15831917.0  0.019117
           2024-10-04   33.50   34.00   33.50   33.80  17712471.0  0.023760
           2024-10-07   33.80   33.85   33.30   33.50  13191324.0  0.019706
           2024-10-08   33.25   33.45   32.90   33.05  14236524.0  0.020897
           2024-10-09   32.90   32.95   32.45   32.45  11396786.0  0.018912
...                       ...     ...     ...     ...         ...       ...
9958       2024-10-28  215.00  216.00  204.50  208.00   6201139.0  0.002693
           2024-10-29  208.00  209.50  203.50  207.50   3874921.0  0.004001
           2024-10-30  208.00  210.00  204.00  207.00   3642748.0  0.003805
           2024-11-01  203.00  210.50  201.50  209.00   3131748.0  0.004953
           2024-11-04  210.00  215.00  207.50  211.00   3116502.0  0.003305

[6804 rows x 6 columns]

### Strategy Lake API

In [7]:
data_api_url = ("https://dev-api.ddt-dst.cc/api/plumber/nextron/industry_prod_df")
df = pd.read_parquet(
    data_api_url,
    storage_options={
        "gcp-token": json.dumps(token),
    }
)

### Load benchmark data

In [8]:
benchmark_inst = "0050"
benchmark_data = pd.read_parquet(
    f"{PLUMBER_HOST}stocks/tw/ohlcv",
    storage_options={
        "gcp-token": json.dumps(token),
        "start-date": start_date,
        "end-date": end_date,
        "tickers": benchmark_inst,
    },
)
benchmark_data.index.set_levels(
    pd.to_datetime(benchmark_data.index.levels[1]),
    level=1,
    inplace=True,
)
benchmark_data.rename_axis(index={
    'ticker': 'instrument'
}, inplace=True)

benchmark_data

/tmp/ipykernel_52823/2173060872.py:11: FutureWarning: inplace is deprecated and will be removed in a future version.
  benchmark_data.index.set_levels(


open    high     low   close    volume
instrument datetime                                            
0050       2024-10-01  183.90  184.60  183.35  183.60   9883064
           2024-10-04  184.00  185.30  183.00  183.60   9244059
           2024-10-07  185.80  188.20  185.80  188.15  13993911
           2024-10-08  186.95  187.35  185.50  187.25   9953432
           2024-10-09  189.00  190.20  189.00  189.00  11860866
           2024-10-11  189.05  192.80  189.05  192.15  13197709
           2024-10-14  192.10  192.90  190.95  192.30  10725917
           2024-10-15  193.00  196.60  193.00  196.40  15738243
           2024-10-16  192.45  195.30  191.65  193.00  18604770
           2024-10-17  194.00  194.10  191.90  193.30   9936801
           2024-10-18  199.30  200.20  197.25  198.10  33596096
           2024-10-21  199.05  199.95  198.00  198.05  13716495
           2024-10-22  197.20  197.30  195.35  197.30   8995727
           2024-10-23  196.70  196.90  195.65  196.00  10050673
           2024-10-24  196.00  196.70  194.50  194.80   8642395
           2024-10-25  195.00  196.55  195.00  196.40  12961246
           2024-10-28  197.90  198.15  194.95  196.05  11929580
           2024-10-29  193.25  193.25  190.20  191.50  26008937
           2024-10-30  192.95  194.05  191.50  191.80  14531256
           2024-11-01  186.15  190.55  185.95  190.50  21072030
           2024-11-04  191.05  193.20  190.80  192.95   7681874

### Initialize Tidal
1. Initialize Tidal object
2. Add Quote data (pd.DataFrame)
3. Set strategy object (td.BaseStrategy)
4. Add metric objects (td.BaseMetic)

In [9]:
# Initialize Tidal object
tidal = td.Tidal(init_cash=50000000, slip_ticks=1, stock_config=td.StockConfig.TW, load_configs=True, reqMem="1000Mi", ignore_volume_size=True)

# Add quote data
tidal.add_quote(quote_data)

# Set strategy object
tidal.set_strategy(td.strategy.TopkDropout(5, 1, 'new'))

# Set metric objects
tidal.add_metric(td.metric.AccountInfo())
tidal.add_metric(td.metric.AdditionalInfo())
tidal.add_metric(td.metric.PositionInfo())
tidal.add_metric(td.metric.Portfolio(benchmark_data.loc[benchmark_inst]))

### Stock Config

In [10]:
tidal.exchange.stock_config

InstConfig {Margin:0.0, Tick Size:0.002, Tick Value:0.002, Trade Unit:1000, Commission:0.0004275, Min Commission:20.0, Transaction Tax:0.003}

### Instrument Configs

In [11]:
tidal.exchange.inst_configs

{'TX': InstConfig {Margin:184000.0, Tick Size:1.0, Tick Value:200.0, Trade Unit:1, Commission:40.0, Min Commission:0.0, Transaction Tax:2e-05},
 'MTX': InstConfig {Margin:46000.0, Tick Size:1.0, Tick Value:50.0, Trade Unit:1, Commission:20.0, Min Commission:0.0, Transaction Tax:2e-05},
 'NQ': InstConfig {Margin:16500.0, Tick Size:0.25, Tick Value:5.0, Trade Unit:1, Commission:1.85, Min Commission:0.0, Transaction Tax:0.0}}

### Config Modification

In [12]:
# Set commission to 77% off
tidal.exchange.set_stock_config(commission=0.001425 * 0.23)
tidal.exchange.stock_config

InstConfig {Margin:0.0, Tick Size:0.002, Tick Value:0.002, Trade Unit:1000, Commission:0.00032775, Min Commission:20.0, Transaction Tax:0.003}

### Add New Instrument Config

In [13]:
# Add MGC config
tidal.exchange.set_config(instrument='MGC', margin=787., tick_size=0.1, tick_value=1., trade_unit=1, commission=2., min_commission=0., transaction_tax=0.)
tidal.exchange.inst_configs

{'TX': InstConfig {Margin:184000.0, Tick Size:1.0, Tick Value:200.0, Trade Unit:1, Commission:40.0, Min Commission:0.0, Transaction Tax:2e-05},
 'MTX': InstConfig {Margin:46000.0, Tick Size:1.0, Tick Value:50.0, Trade Unit:1, Commission:20.0, Min Commission:0.0, Transaction Tax:2e-05},
 'NQ': InstConfig {Margin:16500.0, Tick Size:0.25, Tick Value:5.0, Trade Unit:1, Commission:1.85, Min Commission:0.0, Transaction Tax:0.0},
 'MGC': InstConfig {Margin:787.0, Tick Size:0.1, Tick Value:1.0, Trade Unit:1, Commission:2.0, Min Commission:0.0, Transaction Tax:0.0}}

### Start Backtesting

In [14]:
tidal.backtest()
logs = tidal.get_logs()
for log in logs:
    print(log)

Tidal Backtesting: 100%|██████████| 21/21 [00:00<00:00, 24.82it/s, cash=5.32e+5, pnl=-1.8e+5, position_cost=4.62e+7, value=4.65e+7] 


### Metric - AccountInfo

In [15]:
account_info = tidal.metrics["AccountInfo"].report
print(account_info)

                    cash     trade_cost  position_cost  slip_cost        pnl  \
datetime                                                                       
2024-10-01  5.000000e+07       0.000000            0.0        0.0        0.0   
2024-10-04  3.177497e+06   15341.047477     46807162.4   -62387.6 -1039712.4   
2024-10-07  6.162708e+05   50518.779334     49202560.4    30710.4  -750560.4   
2024-10-08  6.101058e+05   93250.800918     48936262.4    82643.4 -1096762.4   
2024-10-09  5.165515e+05  124405.146018     48283234.4     6693.4  -798334.4   
2024-10-11  6.953585e+05  159044.059237     48121760.4  -121752.6  -910660.4   
2024-10-14  5.204868e+05  189334.203029     47960162.4   -67671.0  -958662.4   
2024-10-15  4.163109e+05  222720.092856     47876136.4    89619.0 -1434936.4   
2024-10-16  4.307682e+05  254243.616554     47260365.6   113788.2 -1001365.6   
2024-10-17  4.751684e+05  282719.383913     46566063.6   121912.2   125336.4   
2024-10-18  5.710853e+05  310855.513284 

### Metric - PositionInfo

In [16]:
position_df = tidal.metrics["PositionInfo"].report
print(position_df)

                       quantity    price   commission  slip_cost       pnl  \
instrument datetime                                                          
1102       2024-10-09  181000.0  47.4000  2811.898350     9050.0 -144800.0   
           2024-10-11  181000.0  47.4000  2811.898350     9050.0 -253400.0   
           2024-10-16  187000.0  45.8916  2812.661745  -123120.8  -35829.2   
           2024-10-17  187000.0  45.8916  2812.661745  -123120.8  207270.8   
           2024-10-18  187000.0  45.8916  2812.661745  -123120.8  319470.8   
...                         ...      ...          ...        ...       ...   
9904       2024-10-30  258000.0  36.4728  3084.121732   -84417.6  342417.6   
           2024-11-01  258000.0  36.4728  3084.121732   -84417.6  381117.6   
           2024-11-04  258000.0  36.4728  3084.121732   -84417.6  497217.6   
9933       2024-10-04  201000.0  47.4000  3122.605350    50250.0 -170850.0   
9938       2024-11-01   97000.0  80.1600  2548.426680   -62080.0

In [17]:
pi_report = tidal.metrics['PositionInfo'].report
pi_report.iloc[pi_report.index.get_level_values('datetime') == '2022-06-14']

,,quantity,price,commission,slip_cost,pnl,now_price
instrument,datetime,,,,,,


### Metric - AdditionalInfo

In [18]:
tidal.metrics['AdditionalInfo'].report

new
instrument datetime            
1101       2024-10-01  0.019117
           2024-10-04  0.023760
           2024-10-07  0.019706
           2024-10-08  0.020897
           2024-10-09  0.018912
...                         ...
9958       2024-10-28  0.002693
           2024-10-29  0.004001
           2024-10-30  0.003805
           2024-11-01  0.004953
           2024-11-04  0.003305

[6804 rows x 1 columns]

### Strategy Lake Submit

In [19]:
group = dict(zip(df["coid"].astype(str), df["tejind4_c"]))
submit_lake_backtest_result = tidal.submit_lake_monitor(account_info=account_info, position_df=position_df, benchmark_info=benchmark_data, group=group,lake_env="dev",strategy_id=19)
print(submit_lake_backtest_result)

update lake monitoring data success


### Trade Report

In [20]:
tidal.trade_report

,win_num,lose_num,trade_num,pos_num,win_rate,profit,loss,trade_cost,pnl
instrument,,,,,,,,,
9904,1.0,0.0,1.0,1.0,1.000000,497217.600000,0.000000e+00,9.409982e+06,4.972176e+05
9938,0.0,1.0,1.0,0.0,0.000000,0.000000,-5.936012e+04,2.832012e+04,-5.936012e+04
2707,1.0,1.0,2.0,1.0,0.500000,34143.000000,-1.508908e+05,1.149361e+07,-1.167478e+05
9933,0.0,1.0,1.0,0.0,0.000000,0.000000,-1.650426e+05,3.439264e+04,-1.650426e+05
8358,0.0,1.0,1.0,0.0,0.000000,0.000000,-2.980768e+05,2.780679e+04,-2.980768e+05
2233,1.0,2.0,3.0,0.0,0.333333,100392.925704,-4.607797e+05,1.043838e+05,-3.603868e+05
1102,2.0,2.0,4.0,1.0,0.500000,213778.047628,-5.850750e+05,8.676728e+06,-3.712969e+05
2912,0.0,1.0,1.0,1.0,0.000000,0.000000,-7.623520e+05,9.194352e+06,-7.623520e+05
3455,1.0,4.0,5.0,1.0,0.200000,17280.163677,-8.179405e+05,7.674451e+06,-8.006603e+05


### Traded instruments

In [21]:
tidal.account.trades.keys()

dict_keys(['9933', '2912', '9904', '3455', '3227', '2233', '2707', '1102', '8358', '9938'])

### Trading History

In [22]:
# tidal.account.trades['3015']

### Plot chart by using Plotly

In [23]:
# tidal.analyzer.inst_chart(instrument='3015', metric_name='AdditionalInfo', plot_type=td.PlotType.LINE, scale=1.0)

### Tidal Dashboard

In [24]:
tidal.tdboard()

 * Serving Flask app 'tidal.tdboard'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:37859
 * Running on http://10.136.14.60:37859
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:10.0.10.82 - - [05/Nov/2024 10:54:38] "GET / HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [05/Nov/2024 10:54:38] "GET /static/js/main.d754b0a3.js HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [05/Nov/2024 10:54:39] "GET /static/css/main.bf4d504b.css HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [05/Nov/2024 10:54:40] "GET /images/Tidal_Logo_white.png HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [05/Nov/2024 10:54:40] "GET /api/quote/inst_list HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [05/Nov/2024 10:54:40] "GET /api/metric/metric_list HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [05/Nov/2024 10:54:40] "GET /api/trade/trade_report HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [05/Nov/2024 10:54:

In [ ]:
tidal.account.position_history